In [19]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/mistra/experiment-6/checkpoint-50")
model = AutoModelForCausalLM.from_pretrained("/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/mistra/experiment-6/checkpoint-50", load_in_4bit=True, device_map="auto")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model1 = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", load_in_4bit=True, device_map="auto")
tokenizer1 = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [1]:
from datasets import load_dataset

dataset = load_dataset("thu-coai/esconv")

Generating train split:   0%|          | 0/910 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/195 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/195 [00:00<?, ? examples/s]

In [2]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 910
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 195
    })
    test: Dataset({
        features: ['text'],
        num_rows: 195
    })
})


In [17]:
import json
data = json.loads(dataset['test'][1]['text'])

In [21]:
print(data['dialog'][1]['text'])

Hai. My name is Tom and i really need your help.


In [64]:
print(len(data['dialog']))

29


In [52]:
messages = [{
    "role": "user",
    "content": data['dialog'][1]['text']
}]

In [ ]:
from evaluate import load
perplexity = load("perplexity", module_type="metric")
perplexity.compute(model_id="clm-gpu/checkpoint-138000",
                             add_start_token=False,
                             predictions=input_texts, 
                             local_file_only=True)

In [104]:
import torch
from tqdm import tqdm

max_length = 1024
stride = 128

#encodings = tokenizer.encode_plus(data['dialog'][1]['text'], return_tensors="pt").to('cuda')
tokenizer.pad_token = tokenizer.eos_token
seq_len = 1024

nlls = []
prev_end_loc = 0
messages = []
i = 1
while i<29:
    for begin_loc in tqdm(range(0, seq_len, stride)):
        messages.append({"role": "user", "content":  data['dialog'][i]['text']})
        i+=2
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        #input_ids = encodings.input_ids[:, begin_loc:end_loc].to('cuda')
        input_ids = tokenizer.apply_chat_template(messages, tokenize=True, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            
            outputs = model.generate(
                input_ids=input_ids,
                labels=target_ids,
                pad_token_id=tokenizer.eos_token_id,
                max_new_tokens=128,
                do_sample=True,
                temperature=0.5,
                top_k=50,
                top_p=0.95
        )
            val = (tokenizer.decode(outputs[0], skip_special_tokens=True)[0].split("[/INST]")[-1])
            outputs = model(input_ids, labels=target_ids)
            print(outputs.logits.shape)
            print(torch.log(torch.softmax(outputs.logits,dim=-1)))
            #decoded_output = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outputs.logits.argmax(-1)]
            #print(tokenizer.batch_decode(outputs, skip_special_tokens=True))
            #print(tokenizer.batch_decode(outputs, skip_special_tokens=True))
            # loss is calculated using CrossEntropyLoss which averages over valid labels
            # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
            # to the left by 1.
            #print(outputs)
            neg_log_likelihood = outputs.loss
            print(neg_log_likelihood.shape)

        nlls.append(neg_log_likelihood)
        print(len(nlls))
        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    ppl = torch.exp(torch.stack(nlls).mean())
    print(ppl)
    messages.append({"role": "assistant", "content": val})

  0%|          | 0/8 [00:01<?, ?it/s]


torch.Size([1, 22, 32000])
tensor([[[-20.7714, -20.7558, -15.5265,  ..., -19.4121, -18.4629, -19.0976],
         [-17.8636, -17.0862, -13.2464,  ..., -13.9007, -15.0784, -13.7542],
         [-18.5422, -17.4162, -21.6535,  ..., -18.3918, -20.6574, -17.1051],
         ...,
         [-17.1520, -14.5641, -21.5261,  ..., -18.7692, -23.4860, -22.5856],
         [-16.5943, -12.8658, -13.8658,  ..., -18.4012, -22.2466, -22.1705],
         [-21.3237, -19.3820, -17.1049,  ..., -22.3432, -22.0288, -22.6460]]],
       device='cuda:1')
torch.Size([])
1
tensor(18.9341, device='cuda:1')


  0%|          | 0/8 [00:03<?, ?it/s]


torch.Size([1, 77, 32000])
tensor([[[-20.8303, -20.8069, -15.5528,  ..., -19.4827, -18.5315, -19.1682],
         [-17.7829, -16.9860, -13.2477,  ..., -13.9079, -15.0290, -13.8258],
         [-18.6158, -17.4591, -21.6451,  ..., -18.6217, -20.8560, -17.3389],
         ...,
         [-14.8625, -12.6945, -12.4992,  ..., -16.4640, -22.5051, -20.9992],
         [-16.9162, -13.5764, -10.6662,  ..., -19.6781, -22.1193, -21.4406],
         [-16.8728, -14.1267,  -4.2380,  ..., -19.1267, -17.5007, -17.5330]]],
       device='cuda:1')
torch.Size([])
2
tensor(20.6048, device='cuda:1')


  0%|          | 0/8 [00:00<?, ?it/s]


...

torch.Size([1, 280, 32000])
tensor([[[-20.8231, -20.8036, -15.5411,  ..., -19.4794, -18.5282, -19.1669],
         [-17.7807, -16.9565, -13.2163,  ..., -13.8803, -15.0932, -13.8725],
         [-18.6858, -17.5154, -21.5433,  ..., -18.7210, -20.9729, -17.4900],
         ...,
         [-17.3430, -15.0559, -12.1809,  ..., -19.1633, -24.3293, -22.7434],
         [-16.9887, -14.3715, -12.8715,  ..., -20.3838, -21.0966, -20.5036],
         [-17.2837, -13.8276,  -0.0327,  ..., -19.2553, -19.7944, -18.3100]]],
       device='cuda:1')
torch.Size([])
12
tensor(17.1217, device='cuda:1')


  0%|          | 0/8 [00:00<?, ?it/s]


torch.Size([1, 299, 32000])
tensor([[[-20.8052, -20.7778, -15.5241,  ..., -19.4653, -18.5181, -19.1528],
         [-17.7185, -16.8669, -13.1697,  ..., -13.8630, -15.1052, -13.8572],
         [-18.5401, -17.3370, -21.3526,  ..., -18.6143, -20.8565, -17.3277],
         ...,
         [-18.2620, -15.9505, -10.1106,  ..., -19.5608, -24.7161, -23.3821],
         [-17.6131, -15.2005,  -7.5442,  ..., -19.7649, -19.9856, -19.3074],
         [-16.3516, -13.1944,  -0.0928,  ..., -17.4932, -18.2647, -17.2813]]],
       device='cuda:1')
torch.Size([])
13
tensor(16.9094, device='cuda:1')


  0%|          | 0/8 [00:00<?, ?it/s]

torch.Size([1, 310, 32000])
tensor([[[-2.0820e+01, -2.0792e+01, -1.5539e+01,  ..., -1.9480e+01,
          -1.8527e+01, -1.9161e+01],
         [-1.7757e+01, -1.6929e+01, -1.3248e+01,  ..., -1.3873e+01,
          -1.5089e+01, -1.3849e+01],
         [-1.8678e+01, -1.7511e+01, -2.1621e+01,  ..., -1.8691e+01,
          -2.0949e+01, -1.7460e+01],
         ...,
         [-1.6764e+01, -1.3988e+01, -6.9804e+00,  ..., -1.8060e+01,
          -2.2848e+01, -2.1211e+01],
         [-1.7153e+01, -1.4231e+01, -6.0593e+00,  ..., -1.9694e+01,
          -2.0158e+01, -1.9247e+01],
         [-1.7389e+01, -1.4155e+01, -1.2033e-02,  ..., -1.9045e+01,
          -1.9563e+01, -1.8481e+01]]], device='cuda:1')
torch.Size([])
14
tensor(16.6619, device='cuda:1')


In [95]:
import torch
from tqdm import tqdm

max_length = 1024
stride = 128

#encodings = tokenizer.encode_plus(data['dialog'][1]['text'], return_tensors="pt").to('cuda')
tokenizer1.pad_token = tokenizer1.eos_token
seq_len = 1024

nlls = []
prev_end_loc = 0
messages = []
i = 1
while i<29:
    for begin_loc in tqdm(range(0, seq_len, stride)):
        messages.append({"role": "user", "content":  data['dialog'][i]['text']})
        i+=2
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        #input_ids = encodings.input_ids[:, begin_loc:end_loc].to('cuda')
        input_ids = tokenizer1.apply_chat_template(messages, tokenize=True, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            
            outputs = model1.generate(
                input_ids=input_ids,
                labels=target_ids,
                pad_token_id=tokenizer.eos_token_id,
                max_new_tokens=128,
                do_sample=True,
                temperature=0.5,
                top_k=50,
                top_p=0.95
        )
            val = (tokenizer1.decode(outputs[0], skip_special_tokens=True)[0].split("[/INST]")[-1])
            outputs = model1(input_ids, labels=target_ids)
            #decoded_output = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outputs.logits.argmax(-1)]
            #print(tokenizer.batch_decode(outputs, skip_special_tokens=True))
            #print(tokenizer.batch_decode(outputs, skip_special_tokens=True))
            # loss is calculated using CrossEntropyLoss which averages over valid labels
            # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
            # to the left by 1.
            #print(outputs)
            neg_log_likelihood = outputs.loss

        nlls.append(neg_log_likelihood)

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    ppl = torch.exp(torch.stack(nlls).mean())
    print(ppl)
    messages.append({"role": "assistant", "content": val})

  0%|          | 0/8 [00:02<?, ?it/s]


tensor(455.2411, device='cuda:1')


  0%|          | 0/8 [00:06<?, ?it/s]


tensor(333.4897, device='cuda:1')


  0%|          | 0/8 [00:06<?, ?it/s]


...

tensor(1179.7234, device='cuda:1')


  0%|          | 0/8 [00:06<?, ?it/s]


tensor(1296.0780, device='cuda:1')


  0%|          | 0/8 [00:06<?, ?it/s]

tensor(1420.3734, device='cuda:1')


In [93]:
import torch
from tqdm import tqdm

#max_length = model.config.n_positions
stride = 512
text = "Absolutely, building this relationship means being there for each other through thick and thin. Whether it's sharing our joys, navigating challenges, or simply lending an ear, I'm dedicated to fostering a connection grounded in trust, empathy, and genuine care. Together, we can create a safe space where open dialogue and mutual support flourish."
encodings = tokenizer.encode_plus("Of course! I'm here to help and chat with you whenever you need.", return_tensors="pt").to('cuda')
seq_len = encodings.input_ids.size(1)

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to('cuda')
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

100%|██████████| 1/1 [00:00<00:00,  7.56it/s]


In [71]:
print(outputs.logits.shape)

torch.Size([1, 18, 32000])


In [94]:

print(torch.mean(torch.max(torch.softmax(outputs.logits, dim=-1), dim=-1)[0]))

tensor(0.6309, device='cuda:1')


In [84]:

print(torch.max(torch.softmax(outputs.logits, dim=-1),dim=-1))

torch.return_types.max(
values=tensor([[0.4461, 0.4940, 0.7285, 0.4279, 0.1381, 0.9768, 0.9906, 0.7321, 0.3723]],
       device='cuda:1'),
indices=tensor([[22478,   264, 10360,   354,   272,   266, 28127,  7768,    13]],
       device='cuda:1'))


In [92]:

print(torch.mean(torch.max(torch.softmax(outputs.logits, dim=-1), dim=-1)[0]))

tensor(0.6380, device='cuda:1')
